# Introduction To Data Science - Final Project

## Group members:

| Name              | ID       |
|-------------------|----------|
| Pham Dang Son Ha |          |
| Tran Dai Nien     | 21127664 |
| Nguyen Cao Khoi   |          |
|   |  |

## Table of Contents

1. [Data Collection](#data-collection)

2. [Data Preprocessing and Exploration](#data-preprocessing-and-exploration)

3. [Data Modeling](#data-modeling)

4. [Reference](#references)

## Data Collection

## Data Preprocessing and Exploration

## Data modeling

## References